# Irradiated Monkeys
This data anlysis attempts to aply machine learning concepts to analyze the patterns and make predictions related to Rhesus monkeys housed by Wake Forest University and in collaboration with Dr. Thomas Johnson of Colorado State University and Dr. Susan Bailey of Colorado Atate Uiversity. The monkeys were irradiated with differnt doses, types, and frequencies of radiation, and blood analytics were frequently measured after radiation. The health records of the monkeys were also recorded, and all monkeys that aare iving are still being monitored. Thus, in this notebook I will test the following:
- 1.) Can machine learning cluster patterns in monkey response to radiation?
- 2.) Can machine learning make accurate blood analytic predicitons?
- 3.) Can machine learning predict the health statuses of the monkeys?
- 4.) What features contribute most to the end biomarker and prognostic statuses of the monkeys? 

# Data handling

In [1]:
# handling data
import pandas as pd
import numpy as np
import datetime

# visualizing data
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")

# Statistics
from scipy import stats
#import statsmodels.api as sm
import statsmodels.imputation.mice as smi
import scipy.cluster.hierarchy as hac
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.gridspec as gridspec
import math
import random


#SHAP Value stuff
import shap
from shap import TreeExplainer
from shap import GradientExplainer
from shap import DeepExplainer
from shap import KernelExplainer
from shap import SamplingExplainer
from shap import PartitionExplainer

#ML stuff
import catboost
from catboost import CatBoostClassifier, Pool, CatBoostRegressor, CatBoost

#language handling
#import bert


#from mxnet import nd, autograd, gluon
#from mxnet.gluon import nn, rnn
#import mxnet as mx


print("Setup Complete")

Setup Complete


In [2]:
pwd

'/Users/aidanlew'

In [3]:
cd '/Users/aidanlew/Documents'

/Users/aidanlew/Documents


In [4]:
ls

1245.92s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


CMB_applicationn.docx
Cage1HomeCage_UnfCage2_EN_9.9_TrimDLC_resnet50_fam_unfam_WT_1.3Sep13shuffle1_15000_el_filtered.csv
DLC_WT_social_trial_workbook.xlsx
DLC_WT_social_trial_workbook1.xlsx
DLC_WT_social_trial_workbook2.xlsx
DLC_WT_social_trial_workbook3.xlsx
DLC_WT_social_trial_workbook4.xlsx
FLAG.xlsx
GS_yahoo.csv
Supplimentary_Table_3.pdf
TERRA_supp_fig_2.pdf
TERRA_supp_fig_tbles.xlsx
Untitled.ipynb
WT_test_DLC_data/
aidan_amcas_app
aidan_amcas_app.docx
b_sharp_transformed_telomere_data.csv
eoc_introduction_presentation.pptx
feild_of_veiw
flag.png
freevbound_percent.png
full_monkey_data_r2.xlsx
g2_cell_percent_w-ornase_whole.png
g2_cell_percent_w-rnase_whole.png
g2_cell_w-ornase_whole.png
g2_cell_w-rnase_whole.png
g2_terra_w-ornase.png
g2_terra_w-rnase_whole.png
global_DSB_TERRA.xlsx
global_terra_bar.png
h2ax.xlsx
h2ax_foci-cell.png
h2ax_percent.png
phase3_feild_of_veiw
phase3_feild_of_veiw.csv
requirements.txt
stock_data_trial.xlsx
telo_triw-ornase_whole.png
telo_triw-rnase_whole.p

In [22]:
full_monkey_df = pd.read_excel('../Documents/full_monkey_data_r2.xlsx',
                    #sheet_name=1,
                    skiprows = 0,
                    usecols = ['ID', 'Species', 'SAMPLE_DATE', 'IRRAD ', 'IRRAD_DATE', 'DOSE_(Gy)',
       'DAYS_SINCE_IRRAD', 'MONTHS_SINCE_IRRAD', 'MONTHS AT IRRAD', 'DOB',
       'SEX', 'AGE', 'TYPE', 'WBC_(K/uL)', 'RBC_(M/uL)',
       'Hemaglobin_(HGB_g/dL)', 'Hemocrit_(HCT_%)',
       'Mean_corpuscular_volume_(MCV_fL)',
       'Mean_corpuscular_hemaglobin_(MCH_pg)',
       'Mean_corpuscular_hemaglobin_concentration_(MCHC_g/dL)', 'NEUT_(%)',
       'LYMPHOCYTES_(%)', 'MONO_(%)', 'EOS_(%)', 'BASO_(%)', 'ABS_NEUT_(/uL)',
       'ABS_LYMPH_(/uL)', 'ABS_MONO_(/uL)', 'ABS_EOS_(/uL)', 'ABS_BASO_(/uL)',
       'PLATLET_COUNT_(K/uL)'],
                    engine = 'openpyxl',
                    #date_parser=parser
                         )

full_monkey_df.set_index(['ID'])
#full_monkey_df.dropna(axis=0, inplace=True) 

#x = full_monkey_df.columns
full_monkey_df.tail()

Cannot parse header or footer so it will be ignored


,ID,Species,SAMPLE_DATE,IRRAD,IRRAD_DATE,DOSE_(Gy),DAYS_SINCE_IRRAD,MONTHS_SINCE_IRRAD,MONTHS AT IRRAD,DOB,...,LYMPHOCYTES_(%),MONO_(%),EOS_(%),BASO_(%),ABS_NEUT_(/uL),ABS_LYMPH_(/uL),ABS_MONO_(/uL),ABS_EOS_(/uL),ABS_BASO_(/uL),PLATLET_COUNT_(K/uL)
4547,2112,Rhesus-Chinese,2020-09-23 00:00:00,No,NaT,NaN,NaN,NaN,NaN,1999-09-28,...,67.0,6.2,1.1,0.0,1696,4422,409,73,0,289
4548,2112,Rhesus-Chinese,2021-03-04 00:00:00,No,NaT,NaN,NaN,NaN,NaN,1999-09-28,...,56.1,9.1,1.5,0.0,1312,2210,359,59,0,284
4549,2210,Rhesus-Indian,2021-09-07 00:00:00,No,NaT,NaN,NaN,NaN,NaN,1999-01-01,...,26.8,12.3,1.2,0.0,3104,1394,640,62,0,311
4550,2210,Rhesus-Indian,2021-09-20 00:00:00,No,NaT,NaN,NaN,NaN,NaN,1999-01-01,...,36.2,10.9,1.7,0.0,2970,2100,632,99,0,348
4551,2210,Rhesus-Indian,2021-10-07 00:00:00,No,NaT,NaN,NaN,NaN,NaN,1999-01-01,...,16.0,21.0,3.0,0.0,3360,896,1176,168,0,329


# It's looking like I'll need to parse the dateimes... We have 31 variables and 4551 samples, get stoked
Next Steps: 1.) Concat the health data 2.) Make date parser 3.) Viz ztuff

# Vizualization
    1.) See the blood analytics over time for each monkey
    2.) See the ages of the monkeys
        a.) For the absolute age
        b.) For age at irrad
        c.) For time since irrad